# Sentiment140
## Иерархическая классификация

Поскольку корпус твиттов Sentiment140 размечен автоматически с использованием смайлов, в нем будут твитты котрые на самом деле не несут никакой эмоциональной окраски. Чтобы избавиться от этих твиттов и улучшить качество классификации применим иерархическую классификацию (https://habrahabr.ru/post/149605/).

Для этого сначала из корпуса удалим нейтнальные твитты. Поиск нейтральных твиттов будет осуществляться на основе корпуса нейтральных и эмоционально окрашеных слов Liu and Hu opinion lexicon

Загружаем необходимые библиотеки:

In [1]:
import re
import numpy as np
import pandas as pd

Считываем из корпуса только сами твитты и их эмоциональную окраску (колонки 'sentiment', 'tweet'): 0 = negative 4 = positive

In [2]:
df = pd.read_csv('Data/training.1600000.processed.noemoticon.csv', header=None, usecols=[0,5], names=['sentiment', 'tweet'])
df.head(3)

,sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...


In [3]:
df.shape

(1600000, 2)

In [4]:
positive_tweets = df.tweet[df.sentiment==4].tolist()
negative_tweets = df.tweet[df.sentiment==0].tolist()

In [5]:
print len(positive_tweets)
print len(negative_tweets)

800000
800000


Удаляем из твиттов все ссылки, цифры, хэштеги, username и прочий мусор

In [6]:
pos_clean = []
for s in positive_tweets:
    t = ""
    t = re.sub('(http\S+)', "", s)
    t = re.sub('[^a-zA-z]', " ", t)
    t = re.sub('(#\w+)|(@\w+)|(\d+)|(&gt;)|(&lt;)', "", t)
    t = re.sub('^\s+', "", t)
    t = re.sub('\\n', " ", t)
    pos_clean.append(t)

In [7]:
neg_clean = []
for s in negative_tweets:
    t = ""
    t = re.sub('(http\S+)', "", s)
    t = re.sub('[^a-zA-z]', " ", t)
    t = re.sub('(#\w+)|(@\w+)|(\d+)|(&gt;)|(&lt;)', "", t)
    t = re.sub('^\s+', "", t)
    t = re.sub('\\n', " ", t)
    neg_clean.append(t)

---

После загрузки исходного корпуса твиттов и предварительной очистки сообщений начинаем поиск нейтральных твиттов на основе корпуса нейтральных и эмоционально окрашеных слов.

Запуск алгоритма поиска нейтральных твиттов будем делать параллельно на нескольких ядрах процессора для увеличения производительности.

После окончания работы алгоритма очищенные от неййтральных твиитов списки сохраняются в файлы *Pos_clean_tweets.csv* и *Neg_clean_tweets.csv*

## pos_clean

In [9]:
import ipyparallel as ipp

In [10]:
rc = ipp.Client()
rc.ids

[0, 1, 2, 3, 4, 5, 6]

In [11]:
dv = rc[:]
dv

<DirectView [0, 1, 2, 3,...]>

In [22]:
# Указываем сколько твиттов будем использовать 
s = pos_clean

In [23]:
# Разбиваем на равные части список и отправляем его на клиенты 
dv.scatter('sc', s)

<AsyncResult: scatter>

In [24]:
%%px
from nltk.corpus import opinion_lexicon
from nltk.tokenize import treebank

In [25]:
%%time
%%px

Pos_or_Neg = 0
Neutral = 0
x = len(sc)

p = list(opinion_lexicon.positive())
n = list(opinion_lexicon.negative())
tokenizer = treebank.TreebankWordTokenizer()

PosNeg = []

for i in xrange(x):
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sc[i])]
        
    for word in tokenized_sent:
        if word in p:
            Pos_or_Neg += 1 # positive
            PosNeg.append(sc[i])
            break
        elif word in n:
            Pos_or_Neg += 1 # negative
            PosNeg.append(sc[i])
            break
    

Wall time: 24min 24s


In [26]:
print 'Pos_or_Neg =', sum(dv.gather('Pos_or_Neg').result())
print 'Neutral =', len(s) - sum(dv.gather('Pos_or_Neg').result())

Pos_or_Neg = 503659
Neutral = 296341


In [27]:
PosNeg = dv.gather('PosNeg').result()

In [28]:
len(PosNeg)

503659

#### Write to file

In [29]:
with open('Pos_clean_tweets.csv', 'w') as f:
    for s in PosNeg:
        f.write(s + '\n')

#### Read from file

In [40]:
with open('Pos_clean_tweets.csv', 'r') as f:
    my_list = [line.rstrip('\n') for line in f]

## neg_clean

In [30]:
# Указываем сколько твиттов будем использовать 
s = neg_clean

In [31]:
# Разбиваем на равные части список и отправляем его на клиенты 
dv.scatter('sc', s)

<AsyncResult: scatter>

In [32]:
%%px
from nltk.corpus import opinion_lexicon
from nltk.tokenize import treebank

In [33]:
%%time
%%px

Pos_or_Neg = 0
Neutral = 0
x = len(sc)

p = list(opinion_lexicon.positive())
n = list(opinion_lexicon.negative())
tokenizer = treebank.TreebankWordTokenizer()

PosNeg = []

for i in xrange(x):
    tokenized_sent = [word.lower() for word in tokenizer.tokenize(sc[i])]
        
    for word in tokenized_sent:
        if word in p:
            Pos_or_Neg += 1 # positive
            PosNeg.append(sc[i])
            break
        elif word in n:
            Pos_or_Neg += 1 # negative
            PosNeg.append(sc[i])
            break
    

Wall time: 25min 34s


In [34]:
print 'Pos_or_Neg =', sum(dv.gather('Pos_or_Neg').result())
print 'Neutral =', len(s) - sum(dv.gather('Pos_or_Neg').result())

Pos_or_Neg = 550487
Neutral = 249513


In [35]:
PosNeg = dv.gather('PosNeg').result()

In [36]:
len(PosNeg)

550487

#### Write to file

In [37]:
with open('Neg_clean_tweets.csv', 'w') as f:
    for s in PosNeg:
        f.write(s + '\n')